In [24]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("CMS-Refactor").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
spark

In [25]:
phase_1_path = "outputs/final_out.csv"
dac_path = "../CMS-Classic/Data/Public/DAC_NationalDownloadableFile.csv"

phase_1 = spark.read.csv(
    phase_1_path,
    header=True,
    inferSchema=True
)

dac = spark.read.csv(
    dac_path,
    header=True,
    inferSchema=True
)

dac = dac.select([
    "NPI",
    "Facility Name",
    "org_pac_id"
])

In [45]:
from pyspark.sql.functions import *

dac_dropped = dac.dropna()
dac_dropped = dac_dropped.dropDuplicates()

# Convert org_pac_id to 'int'
dac_dropped = dac_dropped.withColumn(
    "org_pac_id",
    floor(dac_dropped.org_pac_id)
)

grouped_df = dac_dropped.groupBy('NPI').agg(
    collect_list("Facility Name").alias("Facility_Names"),
    collect_list("org_pac_id").alias("org_pac_ids")
)

max_length = grouped_df.selectExpr("size(Facility_Names) as size").rdd.max()[0]

for i in range(max_length):
    grouped_df = grouped_df.withColumn(f"Facility_Name_{i+1}", expr(f"Facility_Names[{i}]")).withColumn(f"Org_pac_id_{i+1}", expr(f"org_pac_ids[{i}]"))

grouped_df = grouped_df.drop("Facility_Names", "org_pac_ids")

In [95]:
merged_inner = phase_1.join(
    grouped_df,
    "NPI",
    "inner"
)
merged_inner.count()

8915

In [47]:
merged_left = phase_1.join(
    grouped_df,
    "NPI",
    "left"
)
merged_left.count()

10196

In [48]:
# Rearrange the columns
columns = phase_1.columns[:27]
columns.remove('_c0')
g_cols = grouped_df.columns
g_cols.remove('NPI')
columns.extend(g_cols)
columns.extend(phase_1.columns[27:])

merged_inner = merged_inner.select(columns)
merged_left = merged_left.select(columns)

In [49]:
merged_inner.toPandas().to_csv("outputs/cms_dac_inner.csv")
merged_left.toPandas().to_csv("outputs/cms_dac_left.csv")

In [50]:
grouped_df_c = dac_dropped.groupBy('NPI').agg(
    collect_list("Facility Name").alias("Facility_Names"),
    collect_list("org_pac_id").alias("org_pac_ids")
)

max_length = grouped_df_c.selectExpr("size(Facility_Names) as size")# .rdd.max()

In [51]:
frequency = max_length.groupBy('size').count()

In [85]:
import pandas as pd

frequency_df = frequency.toPandas()

In [86]:
frequency_df = frequency_df.sort_values(by='count', ascending=False).reset_index(drop=True)
frequency_df.columns = ['Facility Count', 'Physician Count']
frequency_df.to_csv('Facility_frequency.csv', index=False)

In [89]:
weighted_avg = (frequency_df['Facility Count'] * frequency_df['Physician Count']).sum() / frequency_df['Physician Count'].sum()
weighted_avg

np.float64(1.1802600221990076)

In [91]:
frequency_df[frequency_df['Physician Count'] == frequency_df['Physician Count'].max()]

,Facility Count,Physician Count
0,1,1060138


In [93]:
merged_inner.columns

['NPI',
 'Provider Last Name',
 'Provider First Name',
 'Provider Middle Name',
 'Provider First Line Business Mailing Address',
 'Provider Business Mailing Address City Name',
 'Provider Business Mailing Address State Name',
 'Provider Business Mailing Address Postal Code',
 'Primary_Classification',
 'Primary_Specialization',
 'Definition11',
 'Notes12',
 'Display Name13',
 'Section14',
 'Secondary_Classification',
 'Secondary_Specialization',
 'Definition17',
 'Notes18',
 'Display Name19',
 'Section20',
 'Tertiary_Classification',
 'Tertiary_Specialization',
 'Definition23',
 'Notes24',
 'Display Name25',
 'Section26',
 'Facility_Name_1',
 'Org_pac_id_1',
 'Facility_Name_2',
 'Org_pac_id_2',
 'Facility_Name_3',
 'Org_pac_id_3',
 'Facility_Name_4',
 'Org_pac_id_4',
 'Facility_Name_5',
 'Org_pac_id_5',
 'Facility_Name_6',
 'Org_pac_id_6',
 'Facility_Name_7',
 'Org_pac_id_7',
 'Facility_Name_8',
 'Org_pac_id_8',
 'Facility_Name_9',
 'Org_pac_id_9',
 'Facility_Name_10',
 'Org_pac_id_10'